<a href="https://colab.research.google.com/github/MarkNgendo/MTN-Cote-d-Ivoire-Project/blob/main/MTN_Cote_d'Ivoire_Product_Usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PROBLEM STATEMENT


Currently MTN Cote d'Ivoire would like to upgrade its technology infrastructure for its mobile users in Ivory Coast. Studying the given dataset, how does MTN Cote d'Ivoire go about the upgrade of its infrastructure strategy within the given cities?

For analysis, the following questions were investigated:

Which cities have the highest user traffic based on provided data? Which MTN products are the most frequently used? Which cities bring in the highest profits per product based on provided data? Which are the busiest hours of the day in terms of user traffic?

## DATA RELEVANCE
Below is a list of datasets that we shall be working on:


*  cells_geo_description.xlsx
*   cells_geo.csv 
*   CDR_description.xlsx 
*   CDR 20120507 
*   CDR 20120508 
*   CDR 20120509 










## Importing Libraries





In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Loading Datasets

In [2]:
cells = pd.read_csv('cells_geo.csv', delimiter=';')
cells.head()

,Unnamed: 0,VILLES,STATUS,LOCALISATION,DECOUPZONE,ZONENAME,LONGITUDE,LATITUDE,REGION,AREA,CELL_ID,SITE_CODE
0,0,ADJAME,In Service,ABIDJAN,"""Abidjan_EST",AGBANNEW,-4.000667,5.368194,LAGUNE,NaN,fbb2e75af3,0bb492ef17
1,1,ADJAME,In Service,ABIDJAN,"""Abidjan_EST",AGBANNEW,-4.000667,5.368194,LAGUNE,NaN,1502501d78,0bb492ef17
2,2,ADJAME,In Service,ABIDJAN,"""Abidjan_EST",AGBANNEW,-4.000667,5.368194,LAGUNE,NaN,0c2e182a0c,0bb492ef17
3,3,ASSINIE,In Service,INTERIEUR,Grand-EST,"ASSINIE""",-3.488467,5.158761,SUD COMOE,NaN,69bdfc69fd,8a1e6ce31f
4,4,ASSINIE,In Service,INTERIEUR,Grand-EST,"ASSINIE""",-3.488467,5.158761,SUD COMOE,NaN,7a1029d2ab,8a1e6ce31f


In [ ]:
geo = pd.read_excel('cells_geo_description.xlsx')
geo

,Column name,Description,Format
0,VILLES,City,String
1,STATUS,In Service or not,String
2,LOCALISATION,in ABIDJAN or not,String
3,DECOUPZONE,Geographical Zone,String
4,ZONENAME,Name of Zone,String
5,LONGITUDE,Longitude,Float
6,LATITUDE,Latitude,Float
7,REGION,Region,String
8,AREA,Area,String
9,CELL_ID,ID of the cell,String


In [ ]:
cdr = pd.read_excel('CDR_description.xlsx')
cdr

,Column name,Description,Format
0,PRODUCT,Voice or SMS,String
1,VALUE,Billing price,Integer
2,DATE_TIME,Time in format yyyy-MM-dd hh:mm:ss.0,String
3,CELL_ON_SITE,Which cell in the site was used (not needed here),Integer
4,DW_A_NUMBER_INT,Anonymized phone number of the person for whic...,String
5,DW_B_NUMBER_INT,Anonymized phone number of the counterparty,String
6,COUNTRY_A,Country of party A (useless here),String
7,COUNTRY_B,Country of party B (useless here),String
8,CELL_ID,ID of the cell,String
9,SITE_ID,ID of the SITE,String


In [5]:
set1 = pd.read_csv('Telcom_dataset.csv')
set1.rename(columns={'PRODUTC': 'PRODUCT'}, inplace=True)
set1.head()

,PRODUCT,VALUE,DATETIME,CELL_ON_SITE,DW_A_NUMBER_INT,DW_B_NUMBER_INT,COUNTRY_A,COUNTRY_B,CELL_ID,SITE_ID
0,Voice,0,2012-05-06 23:04:37.0,1,49fae048b547780,1a49f8b0ded9de6,19e2e,19e2e,/TJNe+Mmtu,/TJNe+Mmtu
1,sms,0,2012-05-06 23:05:25.0,0,7a54a6eaee19e65,93343ba6e41af95,19e2e,19e2e,/+cKZKIp41,/+cKZKIp41
2,sms,25,2012-05-06 23:05:41.0,0,6008887b0207d82,7aacc9709ce0a20,19e2e,19e2e,/u0FSD+ahi,/u0FSD+ahi
3,Voice,56,2012-05-06 23:05:42.0,1,7370032e20f4e96,6af4a60e74c91d3,19e2e,19e2e,+cNeJzsTp3,+cNeJzsTp3
4,Voice,14,2012-05-06 23:06:29.0,1,966ba4656e38e62,02aa736ee555d5c,19e2e,19e2e,051KhYDCpv,051KhYDCpv


In [4]:
set2 = pd.read_csv('Telcom_dataset2.csv')
set2.head()

,PRODUCT,VALUE,DATE_TIME,CELL_ON_SITE,DW_A_NUMBER,DW_B_NUMBER,COUNTRY_A,COUNTRY_B,CELL_ID,SITE_ID
0,sms,0,2012-05-07 23:02:06.0,2,32255675c3b6fe3,1aa7ae01386e9fb,19e2e,19e2e,+854AcBQT2,+854AcBQT2
1,Voice,0,2012-05-07 23:03:44.0,1,ff16d6ac74c1c64,87d87309c15a8c9,19e2e,19e2e,+854AcBQT2,+854AcBQT2
2,sms,0,2012-05-07 23:04:06.0,0,98d9f2fb5ff624b,93343ba6e41af95,19e2e,19e2e,+laSrk7g6q,+laSrk7g6q
3,Voice,37,2012-05-07 23:04:48.0,1,3f9c8a33ec7be1f,7d05a38f89c49d8,19e2e,19e2e,/xvRrCVKoQ,/xvRrCVKoQ
4,sms,25,2012-05-07 23:04:58.0,1,7bcccc03adf3864,56a5d5430d2fa86,19e2e,19e2e,+bO+qkSonO,+bO+qkSonO


In [7]:
set3 = pd.read_csv('Telcom_dataset3.csv')
set3.rename(columns= {"SIET_ID": "SITE_ID"}, inplace=True)
set3.head()

,PRODUCT,VALUE,DATE_TIME,CELL_ON_SITE,DW_A_NUMBER_INT,DW_B_NUMBER_INT,COUNTRY_A,COUNTRY_B,CELLID,SITE_ID
0,Voice,61,2012-05-08 23:01:28.0,2,966f68353622feb,ffd53a3bb2db6f1,19e2e,19e2e,+38u2u/rfx,+38u2u/rfx
1,sms,0,2012-05-08 23:01:31.0,1,a0a597384c612d7,cf3b218f1dde792,19e2e,19e2e,+Fs4mTvdKx,+Fs4mTvdKx
2,Voice,10,2012-05-08 23:02:59.0,1,5441a9a58c97132,52f8b299515b9b7,19e2e,19e2e,+ZaaZozy/+,+ZaaZozy/+
3,sms,0,2012-05-08 23:03:04.0,0,8ab8b9ed6989f61,e09e736e46c5c3d,19e2e,19e2e,+cNeJzsTp3,+cNeJzsTp3
4,Voice,22,2012-05-08 23:03:16.0,4,52662a62b199334,2186273faab4602,19e2e,19e2e,/eeMklgyJA,/eeMklgyJA


The next step involves data exploration before cleaning and eventual analysis can be done.

### Possible problems to be addressed

In the cells dataframe, the 'Abidjan_EST' entry has unintended quotations from the source. Also, since each Telecom dataset represent specific dates, the date-time column needs to only represent time by itself. As such, it needs to be split and renamed. In the cells df, the Unamed column needs to be dropped as it serves no purpose.



## DATA CLEANING

### Uniformity

In [8]:
cells.rename(columns={'VILLES': 'City'}, inplace=True)
cells.replace(to_replace='"Abidjan_EST', value='Abidjan_EST', inplace=True)
cells.drop(['Unnamed: 0'], axis=1, inplace=True)
cells.head()

,City,STATUS,LOCALISATION,DECOUPZONE,ZONENAME,LONGITUDE,LATITUDE,REGION,AREA,CELL_ID,SITE_CODE
0,ADJAME,In Service,ABIDJAN,Abidjan_EST,AGBANNEW,-4.000667,5.368194,LAGUNE,NaN,fbb2e75af3,0bb492ef17
1,ADJAME,In Service,ABIDJAN,Abidjan_EST,AGBANNEW,-4.000667,5.368194,LAGUNE,NaN,1502501d78,0bb492ef17
2,ADJAME,In Service,ABIDJAN,Abidjan_EST,AGBANNEW,-4.000667,5.368194,LAGUNE,NaN,0c2e182a0c,0bb492ef17
3,ASSINIE,In Service,INTERIEUR,Grand-EST,"ASSINIE""",-3.488467,5.158761,SUD COMOE,NaN,69bdfc69fd,8a1e6ce31f
4,ASSINIE,In Service,INTERIEUR,Grand-EST,"ASSINIE""",-3.488467,5.158761,SUD COMOE,NaN,7a1029d2ab,8a1e6ce31f


Eliminating redundant columns in each dataset

In [9]:
eliminate = ['CELL_ON_SITE', 'COUNTRY_A', 'COUNTRY_B']
set1.drop(eliminate, axis=1,inplace=True)
set2.drop(eliminate, axis=1, inplace=True)
set3.drop(eliminate, axis=1, inplace=True)

### Completeness

In [10]:
cells.isnull().sum().sort_values(ascending=False)

STATUS          67
AREA            23
ZONENAME         6
SITE_CODE        0
CELL_ID          0
REGION           0
LATITUDE         0
LONGITUDE        0
DECOUPZONE       0
LOCALISATION     0
City             0
dtype: int64

In [14]:
cells.dropna(inplace=True)

In [11]:
set1.isnull().sum().sort_values(ascending=False)

SITE_ID            716
CELL_ID              0
DW_B_NUMBER_INT      0
DW_A_NUMBER_INT      0
DATETIME             0
VALUE                0
PRODUCT              0
dtype: int64

In [15]:
set1.dropna(inplace=True)

In [12]:
set2.isnull().sum().sort_values(ascending=False)

SITE_ID        676
CELL_ID          0
DW_B_NUMBER      0
DW_A_NUMBER      0
DATE_TIME        0
VALUE            0
PRODUCT          0
dtype: int64

In [16]:
set2.dropna(inplace=True)

In [13]:
set3.isnull().sum().sort_values(ascending=False)

SITE_ID            605
CELLID               0
DW_B_NUMBER_INT      0
DW_A_NUMBER_INT      0
DATE_TIME            0
VALUE                0
PRODUCT              0
dtype: int64

In [17]:
set3.dropna(inplace=True)

### Consistency

In [21]:
cells.duplicated().sum()

17

In [22]:
set1.duplicated().sum()

70

In [23]:
set2.duplicated().sum()

62

In [25]:
set3.duplicated().sum()

85

In [26]:
# Dropping Duplicates in datasets

cells.drop_duplicates(inplace=True)
set1.drop_duplicates(inplace=True)
set2.drop_duplicates(inplace=True)
set3.drop_duplicates(inplace=True)

### Merging Datasets

In [29]:
# Merging the three days' datasets

days = pd.concat([set1,set2,set3], axis=0, join='outer', ignore_index=True)
days.head()

,PRODUCT,VALUE,DATETIME,DW_A_NUMBER_INT,DW_B_NUMBER_INT,CELL_ID,SITE_ID,DATE_TIME,DW_A_NUMBER,DW_B_NUMBER,CELLID
0,Voice,0,2012-05-06 23:04:37.0,49fae048b547780,1a49f8b0ded9de6,/TJNe+Mmtu,/TJNe+Mmtu,NaN,NaN,NaN,NaN
1,sms,0,2012-05-06 23:05:25.0,7a54a6eaee19e65,93343ba6e41af95,/+cKZKIp41,/+cKZKIp41,NaN,NaN,NaN,NaN
2,sms,25,2012-05-06 23:05:41.0,6008887b0207d82,7aacc9709ce0a20,/u0FSD+ahi,/u0FSD+ahi,NaN,NaN,NaN,NaN
3,Voice,56,2012-05-06 23:05:42.0,7370032e20f4e96,6af4a60e74c91d3,+cNeJzsTp3,+cNeJzsTp3,NaN,NaN,NaN,NaN
4,Voice,14,2012-05-06 23:06:29.0,966ba4656e38e62,02aa736ee555d5c,051KhYDCpv,051KhYDCpv,NaN,NaN,NaN,NaN


Enhancing the new concatated table

In [30]:
drop = ['DATE_TIME', 'CELLID', 'DW_A_NUMBER', 'DW_B_NUMBER']
days.drop(drop, axis=1, inplace=True)
days.head()

,PRODUCT,VALUE,DATETIME,DW_A_NUMBER_INT,DW_B_NUMBER_INT,CELL_ID,SITE_ID
0,Voice,0,2012-05-06 23:04:37.0,49fae048b547780,1a49f8b0ded9de6,/TJNe+Mmtu,/TJNe+Mmtu
1,sms,0,2012-05-06 23:05:25.0,7a54a6eaee19e65,93343ba6e41af95,/+cKZKIp41,/+cKZKIp41
2,sms,25,2012-05-06 23:05:41.0,6008887b0207d82,7aacc9709ce0a20,/u0FSD+ahi,/u0FSD+ahi
3,Voice,56,2012-05-06 23:05:42.0,7370032e20f4e96,6af4a60e74c91d3,+cNeJzsTp3,+cNeJzsTp3
4,Voice,14,2012-05-06 23:06:29.0,966ba4656e38e62,02aa736ee555d5c,051KhYDCpv,051KhYDCpv


Merging all datasets now and cleaning them

In [31]:
df = pd.merge(days, cells, how='inner', left_on=['SITE_ID'], right_on=['SITE_CODE'])
df.head()

,PRODUCT,VALUE,DATETIME,DW_A_NUMBER_INT,DW_B_NUMBER_INT,CELL_ID_x,SITE_ID,City,STATUS,LOCALISATION,DECOUPZONE,ZONENAME,LONGITUDE,LATITUDE,REGION,AREA,CELL_ID_y,SITE_CODE
0,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1d103ea638ad1f0,5d9dc897f7,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH,24ccdda3d5,1b5540c02d
1,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1d103ea638ad1f0,5d9dc897f7,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH,ebfc58af70,1b5540c02d
2,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1d103ea638ad1f0,5d9dc897f7,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH,ba8ce6e4ef,1b5540c02d
3,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1d103ea638ad1f0,5d9dc897f7,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH,c2545171dc,1b5540c02d
4,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1d103ea638ad1f0,5d9dc897f7,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH,f15bd5483f,1b5540c02d


In [32]:
# Dropping redundant columns

eliminate = ['DW_B_NUMBER_INT', 'CELL_ID_x', 'CELL_ID_y', 'SITE_CODE']
df.drop(eliminate, axis=1, inplace=True)
df.head()

,PRODUCT,VALUE,DATETIME,DW_A_NUMBER_INT,SITE_ID,City,STATUS,LOCALISATION,DECOUPZONE,ZONENAME,LONGITUDE,LATITUDE,REGION,AREA
0,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH
1,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH
2,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH
3,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH
4,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH


## ANALYSIS

In [33]:
# Regions with the highest number of calls

df['REGION'].value_counts().to_frame()

,REGION
LAGUNE,31953
LACS,2408
BAS SASSANDRA,2327
HAUT SASSANDRA,2274
VALLEE DU BANDAMA,1551
SUD BANDAMA,1386
FROMAGER,904
SAVANE,895
WORODOUGOU,792
18 MONTAGNES,687


Taking the top 10 regions with highest traffic:
* LAGUNE               31953
* LACS                  2408
* BAS SASSANDRA         2327
* HAUT SASSANDRA        2274
* VALLEE DU BANDAMA     1551
* SUD BANDAMA           1386
* FROMAGER              904
* SAVANE                895
* WORODOUGOU            792
* 18 MONTAGNES          687


In [40]:
high_regions =['LAGUNE', 'LACS', 'BAS SASSANDRA', 'HAUT SASSANDRA', 'VALLEE DU BANDAMA',
              'SUD BANDAMA', 'SAVANE', 'FROMAGER' 'WORODOUGOU' ,'MONTAGNES']

hr_df = df[df['REGION'].isin(high_regions)]
hr_df.head()

,PRODUCT,VALUE,DATETIME,DW_A_NUMBER_INT,SITE_ID,City,STATUS,LOCALISATION,DECOUPZONE,ZONENAME,LONGITUDE,LATITUDE,REGION,AREA
0,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH
1,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH
2,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH
3,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH
4,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH


In [41]:
# Which Cities are in high traffic zones?

df['ZONENAME'].value_counts()

ZONE C - Abidjan_OUEST     10121
ZONE E - Abidjan_NORD       6626
ZONE B - Grand-SUD          5411
ZONE B - Abidjan_EST        4760
ZONE D - Abidjan_SUD        4602
ZONE D - Grand-OUEST        4233
ZONE A - Abidjan_CENTRE     4121
ZONE A - Grand-NORD         3843
ZONE E - SUD-OUEST          2993
ZONE C - Grand-EST          1745
ZONE D - Abidjan_EST         186
ZONE C - Abidjan_EST          27
ZONE A - Abidjan_OUEST        21
ZONE A - Grand-SUD            18
ZONE B - Grand-EST            18
ZONE B - Abidjan_OUEST        18
ZONE C - Abidjan_SUD          15
ZONE A - Grand-EST             6
ZONE B - Abidjan_SUD           6
ZONE D - Grand-SUD             6
ZONE B - Grand-NORD            3
ZONE B - Abidjan-EST           3
ZONE B - Abidjan_Nord          3
ZONE B - Abidjan_CENTRE        2
Name: ZONENAME, dtype: int64

The top 10 cities in the high traffic zones are:
* ZONE C - Abidjan_OUEST     10121
* ZONE E - Abidjan_NORD       6626
* ZONE B - Grand-SUD          5411
* ZONE B - Abidjan_EST        4760
* ZONE D - Abidjan_SUD        4602
* ZONE D - Grand-OUEST        4233
* ZONE A - Abidjan_CENTRE     4121
* ZONE A - Grand-NORD         3843
* ZONE E - SUD-OUEST          2993
* ZONE C - Grand-EST          1745

In [42]:
high_zones = ['ZONE C - Abidjan_OUEST','ZONE E - Abidjan_NORD', 'ZONE B - Grand-SUD',
              'ZONE B - Abidjan_EST', 'ZONE D - Abidjan_SUD', 'ZONE A - Grand-NORD',
              'ZONE E - SUD-OUEST', 'ZONE C - Grand-EST', 'ZONE D - Abidjan_EST']
hz_df = df[df['ZONENAME'].isin(high_zones)]
hz_df.head()

,PRODUCT,VALUE,DATETIME,DW_A_NUMBER_INT,SITE_ID,City,STATUS,LOCALISATION,DECOUPZONE,ZONENAME,LONGITUDE,LATITUDE,REGION,AREA
0,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH
1,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH
2,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH
3,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH
4,sms,0,2012-05-07 00:00:00.0,8118c3aa53d6c7b,1b5540c02d,BOUAKE,In Service,INTERIEUR,Grand-NORD,ZONE A - Grand-NORD,-5.027139,7.718417,VALLEE DU BANDAMA,CENTRE NORTH


In [43]:
# What is product usage like in these high traffic zones?

usage = hz_df['PRODUCT'].value_counts().to_frame()
usage

,PRODUCT
sms,23505
Voice,16782


In [44]:
#Which of the two is the more profitable product?

profit = hz_df.groupby(['PRODUCT'])['VALUE'].sum()
(profit / profit.sum()) * 100

PRODUCT
Voice    83.390392
sms      16.609608
Name: VALUE, dtype: float64

In [45]:
# Most profitable zone is?

pd.pivot_table(data=hz_df, values='VALUE', index=['PRODUCT', 'ZONENAME'], aggfunc=np.sum)

VALUE
PRODUCT ZONENAME                      
Voice   ZONE A - Grand-NORD     101088
        ZONE B - Abidjan_EST    182138
        ZONE B - Grand-SUD       99114
        ZONE C - Abidjan_OUEST  171186
        ZONE C - Grand-EST       78725
        ZONE D - Abidjan_EST      8196
        ZONE D - Abidjan_SUD     99242
        ZONE E - Abidjan_NORD   108496
        ZONE E - SUD-OUEST       84489
sms     ZONE A - Grand-NORD      13073
        ZONE B - Abidjan_EST     25147
        ZONE B - Grand-SUD       23184
        ZONE C - Abidjan_OUEST   50578
        ZONE C - Grand-EST        7643
        ZONE D - Abidjan_EST       750
        ZONE D - Abidjan_SUD     24312
        ZONE E - Abidjan_NORD    30851
        ZONE E - SUD-OUEST       10231

In [60]:
# Profits in High Trafffic Zones

profit_hz = hz_df.groupby(['PRODUCT'])['VALUE'].sum()

(profit_hz/ profit_hz.sum() * 100).to_frame()

,VALUE
PRODUCT,
Voice,83.390392
sms,16.609608


Most profitable product in high traffic zones is the Voice call.

In [51]:
pd.pivot_table(data=hz_df, values='VALUE', index=['City'], aggfunc=np.sum).sort_values(by='VALUE', ascending=False)

,VALUE
City,
YOPOUGON,198556
COCODY,169678
ABOBO,119236
KOUMASSI,54958
ADJAME,45482
...,...
TIEMELEKRO,0
SONGON,0
NGATTADOLIKRO,0


In [55]:
# Regions with highest frequency of calls

calls = df[df['PRODUCT']== 'Voice']


# Grouping by Region and aggregating by value of calls

calls.groupby(['REGION'])['VALUE'].sum().to_frame().sort_values(by='VALUE', ascending=False)

,VALUE
REGION,
LAGUNE,711553
BAS SASSANDRA,61076
LACS,45489
HAUT SASSANDRA,42275
VALLEE DU BANDAMA,39072
NZI COMOE,29357
SUD BANDAMA,27456
FROMAGER,25661
18 MONTAGNES,25299


Regions with the highest frequency of calls



1.   LAGUNE                                711553
2.   BAS SASSANDRA                         61076
3.   LAC S                                 45489
4.   HAUT SASSANDRA                        42275
5.   VALLEE DU BANDAMA                     39072
6.   NZI COMOE                             29357



In [64]:
# Most profitable cities the company has invested in

city_profits = df.groupby(['City'])[['VALUE']].sum()
city_profits.sort_values(by='VALUE', ascending=False)

,VALUE
City,
YOPOUGON,198556
COCODY,177745
ABOBO,119236
KOUMASSI,71356
TREICHVILLE,65557
...,...
PAKAOUBO,0
OUELLE,0
SIENSOBA,0


## CONCLUSIONS

* The regions with the highest frequency of calls: Lagune, Bas Sassandra, Lac S, Haut Sassandra, Vallee du Bandama.

* The cities inside high traffic zones are: Yopougon, Cocody, Abobo, Koumassi and Adjame.

* The most profitable product offered by the company is the Voice call which accounts for 83% of the profit. This is also consistent in high traffic zones

* Overall, the most profitable cities the company has invested in are: Yopougon, Cocody, Abobo, Koumassi and Treichville.